# --- Day 16: Reindeer Maze ---
https://adventofcode.com/2024/day/16

In [1]:
def getMaze():
    with open("maze.txt") as file:
        return file.read()

In [2]:
from heapq import heappush, heappop

# Define a Node class where information about a current location (coordinate) is held
class Node:
    def __init__(self, coords: tuple[int], parent, direction: str, cost: int=-1):
        self.coords = coords # Coordinates
        self.parent = parent # This is the parent node that this node came from
        self.direction = direction
        self.cost = cost

    # Returns true if state (coordinates) are equal
    def __eq__(self, other):
        if type(self) == type(other):
            return self.coords == other.coords
        return self.coords == other
    
    def __lt__(self, other):
        return self.cost < other.cost
    
    def __repr__(self):
        return f"{self.coords}, {self.direction}, {self.cost}"

# Formatting
maze = getMaze()
maze = maze.split("\n")
rows, cols = len(maze), len(maze[0])

# Find the start and the end node
for i in range(rows):
    for j in range(cols):
        if maze[i][j] == "S":
            start = (i,j)
        if maze[i][j] == "E":
            end = (i,j)

# Search through the maze until we hit the end
def dijkstra(maze, start, end):
    startNode = Node(start, None, "right", 0) # Create a start node with coords

    # Create frontier
    frontier = []
    heappush(frontier, startNode)

    # Create explored 
    explored = dict()
    explored[startNode.coords] = startNode.cost

    # Loop until frontier is empty
    while frontier:
        currentNode = heappop(frontier)

        # If we've made it to the end, then we can return the current node
        if currentNode == end:
            return currentNode
        
        # Check all successors
        currentCoords = currentNode.coords
        newCost = currentNode.cost + 1
        up = Node((currentCoords[0] - 1, currentCoords[1]), parent=currentNode, direction="up", cost=newCost)
        down = Node((currentCoords[0] + 1, currentCoords[1]), parent=currentNode, direction="down", cost=newCost)
        left = Node((currentCoords[0], currentCoords[1] - 1), parent=currentNode, direction="left", cost=newCost)
        right = Node((currentCoords[0], currentCoords[1] + 1), parent=currentNode, direction="right", cost=newCost)

        # Loop through each child
        for child in [up, down, left, right]:
            # If the spot is open or we make it to the end
            if maze[child.coords[0]][child.coords[1]] in [".", "E"]:

                # If there is a direction change, add 1000 to the cost
                if currentNode.direction != child.direction:
                    child.cost += 1000

                # If new coordinate or cheaper coordinate, add to frontier and explored
                if child.coords not in explored.keys() or explored[child.coords] > child.cost:
                    explored[child.coords] = child.cost
                    heappush(frontier, child)

    # If we made it to the end, then we did not find our goal :(
    return None

print(f"Lowest possible reindeer score: {dijkstra(maze, start, end).cost}")

Lowest possible reindeer score: 98416


# --- Part Two ---

In [3]:
import networkx as nx

# Formatting
maze = getMaze()
maze = maze.split("\n")
rows, cols = len(maze), len(maze[0])

# Find the start and the end node
for i in range(rows):
    for j in range(cols):
        if maze[i][j] == "S":
            start = (i,j,">")
        if maze[i][j] == "E":
            end = (i,j)

# Create digraph
mazeGraph = nx.DiGraph()

# Loop through each point in the maze
for row in range(rows):
    for col in range(cols):
        # Get the possible directions the node could move in and the possible parents it can have
        possibleDirections = []
        possibleParents = []
        
        # If the space is open, add relevant edges
        if maze[row][col] != "#":

            # If the current row and column are at the start, just give the start all 4 directions
            if (row, col, ">") == start:
                mazeGraph.add_edge(start, (row - 1, col, "^"), weight=1001)
                mazeGraph.add_edge(start, (row + 1, col, "v"), weight=1001)
                mazeGraph.add_edge(start, (row, col - 1, "<"), weight=1001)
                mazeGraph.add_edge(start, (row, col + 1, ">"), weight=1)
            # Check all surrounding points for possible parents / destinations
            if maze[row - 1][col] != "#":
                possibleDirections.append((row, col, "^"))
                possibleParents.append((row - 1, col, "v"))
            if maze[row + 1][col] != "#":
                possibleDirections.append((row, col, "v"))
                possibleParents.append((row + 1, col, "^"))
            if maze[row][col - 1] != "#":
                possibleDirections.append((row, col, "<"))
                possibleParents.append((row, col - 1, ">"))
            if maze[row][col + 1] != "#":
                possibleDirections.append((row, col, ">"))
                possibleParents.append((row, col + 1, "<"))
            # Add edges to graph from each possible parent to each possible direction
            # (e.g. parents={a,b} directions={c,d} -> [(a,c),(a,d),(b,c),(b,d)])
            for parent in possibleParents:
                for direction in possibleDirections:
                    # If the direction is the end node, just connect to end node, don't bother about direction
                    if (direction[0], direction[1]) == end:
                        mazeGraph.add_edge(parent, end, weight=1)
                    else:
                        # If the directions are different, that means that we're turning, so the weight is 1001
                        if parent[2] != direction[2]:
                            mazeGraph.add_edge(parent, direction, weight=1001)
                        else:
                            mazeGraph.add_edge(parent, direction, weight=1)

# Loop through all shortest paths using networkx and keep track of unique coordinates in a set
allCoords = set()
for i in list(nx.all_shortest_paths(mazeGraph, start, end, weight="weight")):
    for coord in i:
        allCoords.add((coord[0], coord[1]))

print(f"Number of tiles part of at least one best path through the maze: {len(allCoords)}")

Number of tiles part of at least one best path through the maze: 471
